# Create a PISM domain from a polygon based on the BedMachine v5 grid

In [ ]:
from dask.distributed import Client, progress
import numpy as np
import geopandas as gp
import rioxarray
import cartopy
import cartopy.crs as ccrs
import matplotlib.pylab as plt
from shapely.geometry import Polygon

from pism_ragis.domain import get_bounds, create_domain

xr.set_options(keep_attrs=True)


In [ ]:
def create_local_grid(series: gp.GeoSeries, ds: xr.Dataset, buffer: float = 500,
                     base_resolution: int = 150, multipliers: list | np.ndarray = [1, 2, 3, 4, 6, 8, 10, 12, 16, 20, 24, 30]) -> xr.Dataset:
    minx, miny, maxx, maxy = series["geometry"].buffer(buffer).bounds
    max_mult = multipliers[-1]
    ds_coarse = ds.coarsen({"x": max_mult, "y": max_mult}, boundary="pad").mean()
    ll = ds_coarse.sel({"x": minx, "y": miny}, method="nearest")
    ur = ds_coarse.sel({"x": maxx, "y": maxy}, method="nearest")
    if miny > maxy:
        local_ds = ds_coarse.sel({"x": slice(ll["x"], ur["x"]), "y": slice(ll["y"], ur["y"])})
    else:
        local_ds = ds_coarse.sel({"x": slice(ll["x"], ur["x"]), "y": slice(ur["y"], ll["y"])})
    
    x_bnds, y_bnds = get_bounds(local_ds, 
                                base_resolution=base_resolution,
                                multipliers=multipliers)
    grid = create_domain(x_bnds, y_bnds)
    return grid


In [ ]:
# Coordinate Reference System 
crs = "EPSG:3413"

# the base resolution of BedMachine in meters
base_resolution: int = 150

# the resolutions that you want supported:
# 150, 300, 450, 600, 900, 1200, 1500, 1800, 2400, 3000, 3600, and 4500m
multipliers = [1, 2, 4, 8]
max_mult = multipliers[-1]

buffer = 450

ds = xr.open_dataset("/Users/andy/base/pism-ragis/data/dem/BedMachineGreenland-v5.nc")


nx, ny = 110, 100
bm_ds = ds.isel({"x": slice(1000, 1000+nx), "y": slice(2000, 2000+ny)})
bm_ds["grid"] = xr.ones_like(bm_ds["bed"]) * np.arange(ny * nx).reshape(ny, nx)

domains = gp.read_file("/Users/andy/base/pism-ragis/data/basins/gris_test_domains.gpkg").to_crs(crs)

client = Client()
print(f"Open client in browser: {client.dashboard_link}")

domains_scattered = client.scatter([row for _, row in domains.iterrows()])
ds_scattered = client.scatter(bm_ds)

futures = client.map(create_local_grid, domains_scattered, ds=ds_scattered, base_resolution=base_resolution, multipliers=multipliers)
progress(futures)
grids = client.gather(futures)


dr = base_resolution * multipliers[-1]
cartopy_crs = ccrs.NorthPolarStereo(central_longitude=-45, true_scale_latitude=70, globe=None)

fig = plt.figure(figsize=(36, 16))
ax = fig.add_subplot(111, projection=cartopy_crs)
bm_ds["grid"].plot(ax=ax, cmap="copper", add_colorbar=False, alpha=0.5,
                        transform=cartopy_crs)
bm = bm_ds.coarsen({"x": max_mult, "y": max_mult}, boundary="pad").mean()

Y, X = np.meshgrid(bm_ds.y, bm_ds.x)
ax.scatter(X.ravel(), Y.ravel(), 0.05, "k")

for k, row in domains.iterrows():
    grid = grids[k]    
    x_bnds = grid["x_bnds"][0] 
    y_bnds = grid["y_bnds"][0] if grid["y_bnds"][0][0] > grid["y_bnds"][0][1] else grid["y_bnds"][0][::-1]
  
    sub_ds = bm_ds.sel({"x": slice(*x_bnds), "y": slice(*y_bnds)})
    sub_ds["grid"].plot(ax=ax, cmap="Blues", alpha=0.2,
                        extend="both", vmax=ny * nx,
                        add_colorbar=False,
                        transform=cartopy_crs)

    
    xp = np.arange(grid.x_bnds.values[0][0] + dr/2, grid.x_bnds[0][1]-dr/2+dr, dr)
    yp = np.arange(grid.y_bnds.values[0][0] + dr/2, grid.y_bnds[0][1]-dr/2+dr, dr)
    Y, X = np.meshgrid(yp, xp)

    x_point_list = [grid.x_bnds[0][0], grid.x_bnds[0][0], grid.x_bnds[0][1], grid.x_bnds[0][1], grid.x_bnds[0][0]]
    y_point_list = [grid.y_bnds[0][0], grid.y_bnds[0][1], grid.y_bnds[0][1], grid.y_bnds[0][0], grid.y_bnds[0][0]]

    polygon_geom = Polygon(zip(x_point_list, y_point_list))
    polygon = gp.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])
    polygon.to_file(f"domain_{k}.gpkg")
    ax.scatter([grid.x_bnds[0][0], grid.x_bnds[0][0], grid.x_bnds[0][1], grid.x_bnds[0][1]], 
               [grid.y_bnds[0][0], grid.y_bnds[0][1], grid.y_bnds[0][1], grid.y_bnds[0][0]], 25, "k")
    ax.scatter(X.ravel(), Y.ravel())
    gp.GeoSeries(row["geometry"]).plot(ax=ax, color="w", alpha=0.5)
ax.set_title(None)



In [ ]:
dr = base_resolution * multipliers[-1]

fig = plt.figure(figsize=(36, 16))
ax = fig.add_subplot(111, projection=cartopy_crs)
bm_ds["grid"].plot(ax=ax, cmap="copper", add_colorbar=False, alpha=0.5,
                        transform=cartopy_crs)
bm = bm_ds.coarsen({"x": max_mult, "y": max_mult}, boundary="pad").mean()

Y, X = np.meshgrid(bm_ds.y, bm_ds.x)
ax.scatter(X.ravel(), Y.ravel(), 0.05, "k")

for m_id, row in domains.iterrows():
    name = row["SUBREGION1"]
    print(f"Processing basin {m_id}: {name}")
    grid = create_local_grid(row, bm_ds, base_resolution=base_resolution, multipliers=multipliers)
        
    x_bnds = grid["x_bnds"][0] 
    y_bnds = grid["y_bnds"][0] if grid["y_bnds"][0][0] > grid["y_bnds"][0][1] else grid["y_bnds"][0][::-1]
    sub_ds = bm_ds.sel({"x": slice(*x_bnds), "y": slice(*y_bnds)})
    sub_ds["grid"].plot(ax=ax, cmap="Blues", alpha=0.2,
                        extend="both", vmax=ny * nx,
                        add_colorbar=False,
                        transform=cartopy_crs)

    
    xp = np.arange(grid.x_bnds.values[0][0] + dr/2, grid.x_bnds[0][1]-dr/2+dr, dr)
    yp = np.arange(grid.y_bnds.values[0][0] + dr/2, grid.y_bnds[0][1]-dr/2+dr, dr)
    Y, X = np.meshgrid(yp, xp)
    ax.scatter([grid.x_bnds[0][0], grid.x_bnds[0][0], grid.x_bnds[0][1], grid.x_bnds[0][1]], 
               [grid.y_bnds[0][0], grid.y_bnds[0][1], grid.y_bnds[0][1], grid.y_bnds[0][0]], 25, "k")
    ax.scatter(X.ravel(), Y.ravel())
    gp.GeoSeries(row["geometry"]).plot(ax=ax, color="w", alpha=0.5)
ax.set_title(None)

In [ ]:
grids

In [ ]:
gp.GeoDataFrame.from_dict(t.T)

In [ ]:
t

In [ ]:
d

In [ ]:
from pathlib import Path
from scipy.stats.distributions import uniform
import xarray as xr
from pism_ragis.processing import preprocess_config

p = Path("/Volumes/pism/ragis/hindcasts/2025_03_dem/spatial/").glob("ex_*.nc")
ds = xr.open_mfdataset(p,            
                       preprocess=preprocess_config,
                       parallel=True,
                       decode_cf=True,
                       decode_timedelta=True,
                       combine="nested",
                       concat_dim="exp_id",)
last_dem = ds.isel({"time": -1})["usurf"]
diff = ((last_dem.max(dim=["exp_id"]) - last_dem.min(dim=["exp_id"])) / last_dem.mean(dim=["exp_id"])).compute()
fig = diff.plot(vmin=-0.01, vmax=0.01, cmap="RdBu", extend="both")
fig.figure.savefig("rel_diff.png", dpi=600)

In [ ]:
from pathlib import Path
from scipy.stats.distributions import uniform
import xarray as xr
from pism_ragis.processing import preprocess_nc

p = Path("/Volumes/pism/ragis/hindcasts/2023_11_svd/processed_spatial/").glob("usurf_*.nc")
ds = xr.open_mfdataset(p,            
                       preprocess=preprocess_nc,
                       parallel=True,
                       decode_cf=True,
                       decode_timedelta=True,
                       combine="nested",
                       concat_dim="exp_id",)
last_dem = ds.isel({"time": -1})["usurf"]
diff = ((last_dem.max(dim=["exp_id"]) - last_dem.min(dim=["exp_id"])) / last_dem.mean(dim=["exp_id"])).compute()
fig = diff.plot(vmin=-0.01, vmax=0.01, cmap="RdBu", extend="both")
fig.figure.savefig("rel_diff.png", dpi=600)

In [ ]:
import pandas as pd
uq = pd.read_csv("/Users/andy/base/pism-ragis/uq/gris_ragis_dem_w_posterior_lhs_20.csv")

In [ ]:
uq["surface.pdd.factor_ice"] / 910

In [ ]:
diff.plot(vmin=-0.01, vmax=0.01, cmap="RdBu", extend="both")